In [17]:
import pandas as pd
from datasets import Dataset
from transformers import ElectraConfig, ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
import torch
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import random

seed_value = 1999

random.seed(seed_value)
torch.manual_seed(seed_value)
np.random.seed(seed_value)

In [18]:
# Determine the device to run the model on
# If a GPU with CUDA support is available, use it; otherwise, fallback to the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
# Percorsi dei file CSV
train_csv_path = "https://raw.githubusercontent.com/alesmk/HLT/main/data/New%20dataset/BERT/train_tweets_Transformers_new_binary.csv"
eval_csv_path = "https://raw.githubusercontent.com/alesmk/HLT/main/data/New%20dataset/BERT/eval_tweets_Transformers_new_binary.csv"
test_csv_path = "https://raw.githubusercontent.com/alesmk/HLT/main/data/New%20dataset/BERT/test_tweets_Transformers_new_binary.csv"

In [20]:
from datasets import load_dataset, Dataset, DatasetDict, Features, Value
# Caricamento dei dataset
train_df = pd.read_csv(train_csv_path)
eval_df = pd.read_csv(eval_csv_path)
test_df = pd.read_csv(test_csv_path)


train_df = train_df.sample(frac=1).reset_index(drop=True)
eval_df = eval_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)


dataset = DatasetDict()
dataset['train'] = Dataset.from_pandas(train_df)
dataset['test'] = Dataset.from_pandas(test_df)
dataset['validation'] = Dataset.from_pandas(eval_df)
dataset.shape

{'train': (9904, 2), 'test': (2674, 2), 'validation': (2490, 2)}

In [21]:
from sklearn.preprocessing import LabelEncoder
possible_labels = train_df.cyberbullying_type.unique()

le = LabelEncoder()
label_dict = le.fit_transform(possible_labels)

In [22]:
possible_labels

array([0, 1])

In [23]:
# Preprocessing
tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")

In [24]:
def preprocess_function(examples):
    encoded_example=tokenizer(examples["tweet_text"], padding="max_length", truncation=True, max_length=70, return_tensors="pt")
    encoded_example["label"] = label_dict[le.transform(examples["cyberbullying_type"])]
    return encoded_example

In [25]:
encoded_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/9904 [00:00<?, ? examples/s]

Map:   0%|          | 0/2674 [00:00<?, ? examples/s]

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

In [26]:
# Convertire i dataset in tensori PyTorch
encoded_dataset.set_format("torch")

In [27]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# Definizione del modello e dei parametri per la grid search
def model_init(config):

  # Crea il modello con la configurazione personalizzata
  model =ElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator', config=config)
  return model


# Definire i parametri per la grid search
param_grid = {
    'learning_rate': [1e-4, 2e-5, 3e-5, 5e-5],
    'per_device_train_batch_size': [32, 64],
    'hidden_dropout_prob': [0.3, 0.5],
    'attention_probs_dropout_prob': [0.3, 0.5]
}

# Funzione per impostare gli argomenti di addestramento
def train_args(learning_rate, per_device_train_batch_size, epochs):
    training_args = TrainingArguments(
        output_dir="./results",
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_train_batch_size,
        num_train_epochs=epochs,
        load_best_model_at_end=True,  # Carica il miglior modello al termine dell'addestramento
        metric_for_best_model="eval_loss",  # Metric per il miglior modello
    )
    return training_args

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)


In [28]:
class ComputeMetricsCallback(TrainerCallback):
  def __init__(self):
    self.train_accuracies = []
    self.eval_accuracies = []

  def on_epoch_end(self, args, state, control, **kwargs):
    # Calcolare l'accuratezza sul set di addestramento
    train_predictions = trainer.predict(trainer.train_dataset)
    train_accuracy = accuracy_score(train_predictions.label_ids, np.argmax(train_predictions.predictions, axis=1))
    self.train_accuracies.append(train_accuracy)


    # Calcolare l'accuratezza sul set di validazione
    eval_predictions = trainer.predict(trainer.eval_dataset)
    eval_accuracy = accuracy_score(eval_predictions.label_ids, np.argmax(eval_predictions.predictions, axis=1))
    self.eval_accuracies.append(eval_accuracy)

    # Aggiorna i log solo se esiste un log precedente
    if state.log_history:
      state.log_history[-1]['train_accuracy'] = train_accuracy
      state.log_history[-1]['eval_accuracy'] = eval_accuracy
    else:
      state.log_history.append({'train_accuracy': train_accuracy, 'eval_accuracy': eval_accuracy})


  # Funzione di valutazione personalizzata
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': acc,
        'f1': f1,
    }


In [29]:
# Implementazione della Grid Search con Early Stopping

best_params = None
best_score = float('-inf')

for lr in param_grid['learning_rate']:
  for bs in param_grid['per_device_train_batch_size']:
    for hdp in param_grid['hidden_dropout_prob']:
      for adp in param_grid['attention_probs_dropout_prob']:
        training_args = train_args(lr, bs, 10)
        # Crea una configurazione personalizzata con dropout
        config = ElectraConfig.from_pretrained(
            'google/electra-small-discriminator',
             hidden_dropout_prob = hdp,  # Imposta il tasso di dropout
             attention_probs_dropout_prob = adp,  # Imposta il tasso di dropout per l'attenzione
             num_labels=len(possible_labels)
          )

        model = model_init(config)
        model.to(device)

        #forward pass
        outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0).to(device), attention_mask=encoded_dataset['train']['attention_mask'][0].unsqueeze(0).to(device)
          ,labels=encoded_dataset['train'][0]['label'].unsqueeze(0).to(device))

        mycallback = ComputeMetricsCallback()

        trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=encoded_dataset["train"],
          eval_dataset=encoded_dataset["validation"],
          tokenizer=tokenizer,
          compute_metrics=compute_metrics,
          callbacks=[mycallback, early_stopping_callback]
        )

        # Addestrare il modello
        trainer.train()


        # Recupera i log dell'addestramento
        history = trainer.state.log_history
        validation_loss = [log['eval_loss'] for log in history if 'eval_loss' in log]
        min_val_loss = min(validation_loss)
        index_min_loss = validation_loss.index(min_val_loss)
        validation_accuracy = mycallback.eval_accuracies
        score = validation_accuracy[index_min_loss]

        print(f"Score obtained: {score}")

        if score > best_score:
           best_score = score
           best_params = {
            'learning_rate': lr,
            'batch_size': bs,
            'hidden_dropout_prob': hdp,
            'attention_probs_dropout_prob': adp
        }

  print(f"Best Score: {best_score}")
  print(f"Best Params: {best_params}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.301408,0.892771,0.892643
2,0.352100,0.376454,0.900803,0.900706
3,0.352100,0.354647,0.897189,0.896961


Score obtained: 0.8927710843373494


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.450488,0.850602,0.849501
2,0.406300,0.467623,0.868675,0.867856
3,0.406300,0.444107,0.876305,0.875849
4,0.278000,0.417782,0.885141,0.884761
5,0.228700,0.401233,0.896386,0.896115
6,0.228700,0.236470,0.917269,0.917216
7,0.202400,0.287655,0.913655,0.913582
8,0.202400,0.377945,0.899197,0.898962


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.694835,0.532129,0.452786
2,No log,0.755842,0.573896,0.493861
3,No log,0.699534,0.649799,0.616365


Score obtained: 0.5321285140562249


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.687277,0.517269,0.378443
2,No log,0.698211,0.635341,0.616281
3,No log,0.657438,0.651807,0.649546
4,0.610000,0.644902,0.691566,0.691524
5,0.610000,0.624120,0.681124,0.677129
6,0.610000,0.621634,0.687149,0.682223
7,0.535800,0.587364,0.707631,0.704819
8,0.535800,0.591704,0.714859,0.712757
9,0.535800,0.586828,0.717671,0.715283
10,0.519500,0.587910,0.716867,0.714301


Score obtained: 0.7176706827309237
Best Score: 0.9172690763052209
Best Params: {'learning_rate': 0.0001, 'batch_size': 32, 'hidden_dropout_prob': 0.3, 'attention_probs_dropout_prob': 0.5}
